In [2]:
import pandas as pd

merged_merchant_with_categories = pd.read_parquet('../data/curated/merged_merchant_info_with_categories.parquet')

In [3]:
merged_merchant_with_categories.head()

,merchant_abn,category_label,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions
0,10023283211,Furniture and Home Furnishing,0.18,0.0,216.736529,155.285714
21,10142254217,"Media and Technology, Services",4.22,0.0,37.716001,144.571429
42,10165489824,Retail Goods,4.40,0.0,11236.094771,0.454545
53,10187291046,"Automotives, Retail Goods, Services",3.29,0.0,119.526004,16.800000
73,10192359162,Retail Goods,6.33,0.0,462.329450,19.250000


In [4]:
consumer_tbl = pd.read_csv('../data/raw/tbl_consumer.csv', delimiter='|')
userdf = pd.read_parquet('../data/raw/consumer_user_details.parquet')
merged = consumer_tbl.merge(userdf, on='consumer_id', how='left')

In [5]:
test_df = pd.read_parquet('../test/')

In [6]:
merged_with_test = merged.merge(test_df, on='user_id', how='left')
merged_with_test_dropped = merged_with_test.dropna()

In [7]:
# Dropped na since no one has bought from these merchants
merged_with_abn = merged_with_test_dropped.merge(merged_merchant_with_categories, on='merchant_abn', how='left')
merged_with_abn_dropped = merged_with_abn.dropna()

In [8]:
columns_to_drop = ['name', 'address', 'state', 'gender', 'dollar_value', 'order_id', 'order_datetime', 'category_label']

cleaned_merged_with_abn = merged_with_abn_dropped.drop(columns=columns_to_drop)

In [9]:
census_data = pd.read_csv('../data/curated/2021Census_G02_AUST_POA_curated.csv')
census_data = census_data.rename(columns={'POA_CODE_2021': 'postcode'})
census_data.head()
merged_with_census = cleaned_merged_with_abn.merge(census_data, on='postcode', how='left')

print(merged_with_census.head())
merged_with_census.isnull().sum()

   postcode  consumer_id  user_id  merchant_abn  take_rate_value  \
0      6935      1195503        1  2.800049e+10             4.24   
1      6935      1195503        1  8.369064e+10             3.15   
2      6935      1195503        1  9.668077e+10             5.91   
3      6935      1195503        1  4.989171e+10             5.80   
4      6935      1195503        1  6.240384e+10             6.46   

   fraud_probability  average_monthly_revenue  average_monthly_transactions  \
0           0.000000               245.258150                    180.523810   
1           0.000000                88.512653                   1707.238095   
2          29.555245               316.660874                   1482.571429   
3           0.000000                28.889602                  11786.952381   
4           0.000000                34.467794                    184.571429   

   Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
0                 NaN                   

postcode                               0
consumer_id                            0
user_id                                0
merchant_abn                           0
take_rate_value                        0
fraud_probability                      0
average_monthly_revenue                0
average_monthly_transactions           0
Median_age_persons               2244225
Median_mortgage_repay_monthly    2244225
Median_rent_weekly               2244225
Median_tot_hhd_inc_weekly        2244225
dtype: int64

In [10]:
merged_with_census_dropped = merged_with_census.dropna()
merged_with_census_dropped.head()

,postcode,consumer_id,user_id,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
553,2782,179208,2,8.077982e+10,4.71,0.0,34.567393,1735.142857,50.0,2080.0,430.0,1528.0
554,2782,179208,2,2.864764e+10,3.32,0.0,263.829390,23.476190,50.0,2080.0,430.0,1528.0
555,2782,179208,2,3.581349e+10,5.66,0.0,45.885892,182.761905,50.0,2080.0,430.0,1528.0
556,2782,179208,2,3.809009e+10,3.24,0.0,172.158074,2467.428571,50.0,2080.0,430.0,1528.0
557,2782,179208,2,3.897149e+10,3.61,0.0,166.642856,269.761905,50.0,2080.0,430.0,1528.0


In [11]:
aggregated_df = merged_with_census_dropped.groupby('merchant_abn').mean()
aggregated_df = aggregated_df.reset_index()
aggregated_df = aggregated_df.drop(columns=['consumer_id', 'postcode', 'user_id'])

aggregated_df.head()

,merchant_abn,take_rate_value,fraud_probability,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053


In [12]:
merchant_categories = pd.read_parquet('../data/curated/merchant_categories.parquet')
merchant_categories = merchant_categories.drop(columns=['category', 'revenue_rate', 'fraud_probability', 'take_rate_value'])
category_df = aggregated_df.merge(merchant_categories, on='merchant_abn', how='left')



In [131]:
merchant_fraud = pd.read_csv('../data/raw/merchant_fraud_probability.csv')
final_df = category_df.merge(merchant_fraud, on='merchant_abn', how='left')
final_df['fraud_probability_y'] = final_df['fraud_probability_y'].apply(lambda x: 0 if pd.isna(x) else 1)
final_df = final_df.drop(columns = ['order_datetime'])
final_df = final_df.rename(columns={'fraud_probability_x': 'consumer_fraud', 'fraud_probability_y': 'merchant_fraud'})
final_df2 = final_df


In [132]:
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,43.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,43.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,39.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,43.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,43.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [133]:
final_df['Median_age_persons'] = (25 - final_df['Median_age_persons']).abs()
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0


In [134]:
cwgr = pd.read_csv('../data/curated/cwgr.csv')
final_df = final_df.merge(cwgr, on='merchant_abn', how='left')
final_df.head()

,merchant_abn,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr
0,1.002328e+10,0.18,0.0,216.736529,155.285714,18.367701,1544.717295,297.540281,1598.090540,Felis Limited,Furniture and Home Furnishing,0,0.026743
1,1.014225e+10,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0,0.051124
2,1.016549e+10,4.40,0.0,11236.094771,0.454545,14.500000,1735.250000,332.500000,1855.250000,Nunc Sed Company,Retail Goods,0,0.000000
3,1.018729e+10,3.29,0.0,119.526004,16.800000,18.839041,1563.356164,294.119863,1595.914384,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0,0.000167
4,1.019236e+10,6.33,0.0,462.329450,19.250000,18.287926,1595.092879,309.244582,1657.421053,Enim Condimentum PC,Retail Goods,0,0.007906


In [135]:
final_df['bnpl_revenue'] = final_df['average_monthly_revenue'] * final_df['take_rate_value']
final_df.drop(columns=['average_monthly_revenue', 'take_rate_value'], inplace=True)

In [136]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

# Assuming final_df is your DataFrame

# Columns to exclude from standardization
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to standardize
columns_to_standardize = [col for col in final_df.columns if col not in exclude_columns]

# Initialize the StandardScaler
scaler = StandardScaler()

# Apply the scaler to the columns to be standardized
final_df[columns_to_standardize] = scaler.fit_transform(final_df[columns_to_standardize])

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue
0,1.002328e+10,0.0,-0.011175,-0.080343,-0.081199,-0.100735,-0.053952,Felis Limited,Furniture and Home Furnishing,0,0.957301,-0.429217
1,1.014225e+10,0.0,-0.027384,-0.153495,0.052766,-0.025484,0.083467,Arcu Ac Orci Corporation,"Media and Technology, Services",0,2.425892,-0.422301
2,1.016549e+10,0.0,-0.245407,-3.533623,1.081669,1.685615,3.174476,Nunc Sed Company,Retail Goods,0,-0.653557,2.414334
3,1.018729e+10,0.0,-0.220679,0.340493,0.032559,-0.275510,-0.081271,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0,-0.643482,-0.408827
4,1.019236e+10,0.0,-0.216973,-0.151571,0.226256,0.497324,0.690895,Enim Condimentum PC,Retail Goods,0,-0.177348,-0.263005


In [37]:
#final_df['Median_age_persons'] = (1 - final_df['Median_age_persons'])
#final_df.head()

In [157]:
import pandas as pd

# Assuming final_df is your DataFrame

# Define the weights for each column
weights = {
    'merchant_fraud': -6,
    'cwgr': 2.5,
    'bnpl_revenue': 2.5,
    'consumer_fraud': -2.5,
    'average_monthly_transactions': 0.7,
    'Median_tot_hhd_inc_weekly': 0.2,
    'Median_age_persons': -0.15,
    'Median_mortgage_repay_monthly': -0.1,
    'Median_rent_weekly': -0.1
}

# Calculate the score for each row
final_df['score'] = (
    final_df['merchant_fraud'] * weights['merchant_fraud'] +
    (final_df['bnpl_revenue'] - final_df['bnpl_revenue'].mean()) * weights['bnpl_revenue'] +
    final_df['consumer_fraud'] * weights['consumer_fraud'] +
    (final_df['average_monthly_transactions'] - final_df['average_monthly_transactions'].mean()) * weights['average_monthly_transactions'] +
    (final_df['Median_tot_hhd_inc_weekly'] - final_df['Median_tot_hhd_inc_weekly'].mean()) * weights['Median_tot_hhd_inc_weekly'] +
    (final_df['Median_age_persons'] - final_df['Median_age_persons'].mean()) * weights['Median_age_persons'] +
    (final_df['Median_mortgage_repay_monthly'] - final_df['Median_mortgage_repay_monthly'].mean()) * weights['Median_mortgage_repay_monthly'] +
    (final_df['Median_rent_weekly'] - final_df['Median_rent_weekly'].mean()) * weights['Median_rent_weekly'] +
    (final_df['cwgr'] - final_df['cwgr'].mean()) * weights['cwgr']
)

# Display the first few rows of the updated DataFrame to verify
final_df.head()

,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue,score
0,1.002328e+10,0.0,-0.011175,-0.080343,-0.081199,-0.100735,-0.053952,Felis Limited,Furniture and Home Furnishing,0,0.957301,-0.429217,1.331841
1,1.014225e+10,0.0,-0.027384,-0.153495,0.052766,-0.025484,0.083467,Arcu Ac Orci Corporation,"Media and Technology, Services",0,2.425892,-0.422301,5.026798
2,1.016549e+10,0.0,-0.245407,-3.533623,1.081669,1.685615,3.174476,Nunc Sed Company,Retail Goods,0,-0.653557,2.414334,5.118367
3,1.018729e+10,0.0,-0.220679,0.340493,0.032559,-0.275510,-0.081271,Ultricies Dignissim Lacus Foundation,"Automotives, Retail Goods, Services",0,-0.643482,-0.408827,-2.828282
4,1.019236e+10,0.0,-0.216973,-0.151571,0.226256,0.497324,0.690895,Enim Condimentum PC,Retail Goods,0,-0.177348,-0.263005,-1.164208


In [95]:
import pandas as pd

# Assuming final_df is your DataFrame

# Columns to exclude from the calculations
exclude_columns = ['merchant_abn', 'consumer_fraud', 'merchant_fraud', 'category_label', 'name']

# Columns to include in the calculations
columns_to_include = [col for col in final_df.columns if col not in exclude_columns]

# Calculate the average, min, and max for each included column
averages = final_df[columns_to_include].mean()
minimums = final_df[columns_to_include].min()
maximums = final_df[columns_to_include].max()

# Display the results
print("Averages:")
print(averages)
print("\nMinimums:")
print(minimums)
print("\nMaximums:")
print(maximums)

Averages:
average_monthly_transactions    -3.236360e-18
Median_age_persons               1.109262e-15
Median_mortgage_repay_monthly   -5.902311e-16
Median_rent_weekly              -5.825447e-17
Median_tot_hhd_inc_weekly        1.841489e-15
cwgr                            -2.224997e-17
bnpl_revenue                     3.883631e-17
score                           -1.215724e+01
dtype: float64

Minimums:
average_monthly_transactions      -0.245905
Median_age_persons               -14.694258
Median_mortgage_repay_monthly     -9.508990
Median_rent_weekly               -15.304266
Median_tot_hhd_inc_weekly         -9.991727
cwgr                             -10.081493
bnpl_revenue                      -0.431161
score                           -276.510568
dtype: float64

Maximums:
average_monthly_transactions     20.610117
Median_age_persons               16.555520
Median_mortgage_repay_monthly    51.517374
Median_rent_weekly               11.521862
Median_tot_hhd_inc_weekly        19.102601
cwg

In [158]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Sort the DataFrame by the 'score' column in descending order
sorted_df = final_df.sort_values(by='score', ascending=False)

# Select the top 100 merchants
top_100_merchants = sorted_df.head(100)

# Display the top 100 merchants
top_100_merchants.head(20)


,merchant_abn,consumer_fraud,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud,cwgr,bnpl_revenue,score
1580,3.915015e+10,0.0,-0.244582,-6.658601,-0.915618,-2.529919,2.129340,Aliquam Eu Institute,"Automotives, Services",0,-0.653557,14.894342,37.199966
16,1.040454e+10,0.0,-0.244582,-1.301496,-4.791180,-6.617710,-2.390167,Consequat Foundation,"Automotives, Services",0,-0.653557,8.653181,20.685932
971,2.906845e+10,0.0,-0.244582,16.555520,-9.508990,-15.304266,14.394781,Phasellus LLP,"Automotives, Services",0,-0.653557,6.544969,17.434276
2274,5.486013e+10,0.0,-0.243674,-0.089770,-0.067703,0.176417,-0.099028,Laoreet Posuere Enim Limited,"Automotives, Services",0,-0.551304,7.013607,15.967974
762,2.485245e+10,0.0,20.610117,-0.053077,-0.010563,0.005864,0.011693,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0,0.766714,-0.426377,15.288695
2140,5.142505e+10,0.0,-0.245758,11.198415,-5.236717,-5.084789,-8.252972,Sem Corporation,"Automotives, Services",0,-0.653557,7.648369,15.016793
2679,6.414355e+10,0.0,-0.245590,2.567524,-2.606218,-4.539750,-1.431864,Fusce Feugiat Associates,Retail Goods,0,-0.653557,6.599162,14.735193
3728,8.657848e+10,0.0,19.397053,-0.026657,0.018087,0.055301,0.053075,Leo In Consulting,"Automotives, Retail Goods, Services",0,0.760844,-0.418495,14.441084
2683,6.420342e+10,0.0,18.540222,-0.034216,0.024828,0.063371,0.060682,Pede Nonummy Corp.,Furniture and Home Furnishing,0,0.788065,-0.426697,13.890024
2068,4.989171e+10,0.0,17.585418,-0.041761,0.004607,0.029068,0.028975,Non Vestibulum Industries,Furniture and Home Furnishing,0,0.849664,-0.421818,13.388101


In [161]:
# Assuming top_100_merchants is your DataFrame containing the top 100 merchants

# Count the number of merchants with merchant_fraud equal to 1
num_merchants_with_fraud = (top_100_merchants['average_monthly_transactions'] > -0.2).sum()

# Display the result
print(f"Number of merchants with merchant_fraud = 1: {num_merchants_with_fraud}")

Number of merchants with merchant_fraud = 1: 46


In [178]:
import pandas as pd

# Assuming final_df is your DataFrame and the 'score' column has already been calculated

# Define the category labels
categories = ['Retail Goods', 'Media and Technology', 'Furniture and Home Furnishing', 'Services', 'Automotives']

# Initialize variables to store the top 10 merchants for each category
top_10_retail_goods = None
top_10_media_technology = None
top_10_furniture_home_furnishing = None
top_10_services = None
top_10_automotives = None

# Loop through each category and get the top 10 merchants
for category in categories:
    # Filter and sort the DataFrame by the current category and score
    top_10_merchants = final_df[final_df['category_label'].apply(lambda x: category in x)].sort_values(by='score', ascending=False).head(10)
    

    # Store the result in the corresponding variable
    if category == 'Retail Goods':
        top_10_retail_goods = top_10_merchants
    elif category == 'Media and Technology':
        top_10_media_technology = top_10_merchants
    elif category == 'Furniture and Home Furnishing':
        top_10_furniture_home_furnishing = top_10_merchants
    elif category == 'Services':
        top_10_services = top_10_merchants
    elif category == 'Automotives':
        top_10_automotives = top_10_merchants



In [179]:
# Display the top 10 merchants for each category
print("Top 10 merchants in 'Retail Goods':")
print(top_10_retail_goods)
print("\n")

print("Top 10 merchants in 'Media and Technology':")
print(top_10_media_technology)
print("\n")

print("Top 10 merchants in 'Furniture and Home Furnishing':")
print(top_10_furniture_home_furnishing)
print("\n")

print("Top 10 merchants in 'Services':")
print(top_10_services)
print("\n")

print("Top 10 merchants in 'Automotives':")
print(top_10_automotives)
print("\n")

Top 10 merchants in 'Retail Goods':
      merchant_abn  consumer_fraud  average_monthly_transactions  \
762   2.485245e+10             0.0                     20.610117   
2679  6.414355e+10             0.0                     -0.245590   
3728  8.657848e+10             0.0                     19.397053   
1870  4.562922e+10             0.0                     16.197155   
1810  4.434579e+10             0.0                     -0.239683   
1766  4.318652e+10             0.0                     14.227467   
3424  8.032405e+10             0.0                     13.934052   
2635  6.329052e+10             0.0                     12.856061   
4288  9.759036e+10             0.0                     -0.243371   
4351  9.902266e+10             0.0                     -0.242085   

      Median_age_persons  Median_mortgage_repay_monthly  Median_rent_weekly  \
762            -0.053077                      -0.010563            0.005864   
2679            2.567524                      -2.606218  

In [163]:
import pandas as pd

# Assuming top_100_merchants and final_df2 are your DataFrames

# Select only the 'merchant_abn' and 'score' columns from top_100_merchants
top_100_merchants_reduced = top_100_merchants[['merchant_abn', 'score']]

# Merge with final_df2 on 'merchant_abn', keeping only the merchants from top_100_merchants
top_100_merged = pd.merge(top_100_merchants_reduced, final_df2, on='merchant_abn', how='left')

# Display the first few rows of the merged DataFrame to verify
top_100_merged.head(10)

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,3.915015e+10,37.199966,6.99,0.0,38089.939272,1.000000,11.000000,1408.000000,250.000000,1772.000000,Aliquam Eu Institute,"Automotives, Services",0
1,1.040454e+10,20.685932,6.93,0.0,22773.975280,1.000000,17.000000,773.000000,170.000000,1412.000000,Consequat Foundation,"Automotives, Services",0
2,2.906845e+10,17.434276,3.28,0.0,36950.787544,1.000000,37.000000,0.000000,0.000000,2749.000000,Phasellus LLP,"Automotives, Services",0
3,5.486013e+10,15.967974,6.01,0.0,21520.803196,1.600000,18.357143,1546.928571,302.964286,1594.500000,Laoreet Posuere Enim Limited,"Automotives, Services",0
4,2.485245e+10,15.288695,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
5,5.142505e+10,15.016793,5.71,0.0,24582.746982,0.222222,31.000000,700.000000,200.000000,945.000000,Sem Corporation,"Automotives, Services",0
6,6.414355e+10,14.735193,5.88,0.0,20772.116474,0.333333,21.333333,1131.000000,210.666667,1488.333333,Fusce Feugiat Associates,Retail Goods,0
7,8.657848e+10,14.441084,6.43,0.0,35.036088,12984.476190,18.427831,1560.985112,300.593985,1606.615735,Leo In Consulting,"Automotives, Retail Goods, Services",0
8,6.420342e+10,13.890024,2.86,0.0,28.949356,12418.095238,18.419364,1562.089539,300.751919,1607.221653,Pede Nonummy Corp.,Furniture and Home Furnishing,0
9,4.989171e+10,13.388101,5.80,0.0,28.889602,11786.952381,18.410914,1558.776322,300.080589,1604.696067,Non Vestibulum Industries,Furniture and Home Furnishing,0


In [180]:
import pandas as pd

# Assuming final_df2 is your DataFrame and the top 10 DataFrames are already defined

# Function to merge and keep only 'score' column from top merchants DataFrame
def merge_top_merchants(top_merchants_df, final_df2):
    # Select only the 'merchant_abn' and 'score' columns from top_merchants_df
    top_merchants_reduced = top_merchants_df[['merchant_abn', 'score']]
    
    # Merge with final_df2 on 'merchant_abn', keeping only the merchants from top_merchants_df
    merged_df = pd.merge(top_merchants_reduced, final_df2, on='merchant_abn', how='left')
    
    return merged_df

# Merge for each top 10 merchants DataFrame
merged_top_10_retail_goods = merge_top_merchants(top_10_retail_goods, final_df2)
merged_top_10_media_technology = merge_top_merchants(top_10_media_technology, final_df2)
merged_top_10_furniture_home_furnishing = merge_top_merchants(top_10_furniture_home_furnishing, final_df2)
merged_top_10_services = merge_top_merchants(top_10_services, final_df2)
merged_top_10_automotives = merge_top_merchants(top_10_automotives, final_df2)

# Display the first few rows of each merged DataFrame to verify
print("Merged Top 10 Retail Goods:")
print(merged_top_10_retail_goods.head())
print("\n")

print("Merged Top 10 Media and Technology:")
print(merged_top_10_media_technology.head())
print("\n")

print("Merged Top 10 Furniture and Home Furnishing:")
print(merged_top_10_furniture_home_furnishing.head())
print("\n")

print("Merged Top 10 Services:")
print(merged_top_10_services.head())
print("\n")

print("Merged Top 10 Automotives:")
print(merged_top_10_automotives.head())
print("\n")

Merged Top 10 Retail Goods:
   merchant_abn      score  take_rate_value  consumer_fraud  \
0  2.485245e+10  15.288695             2.94             0.0   
1  6.414355e+10  14.735193             5.88             0.0   
2  8.657848e+10  14.441084             6.43             0.0   
3  4.562922e+10  12.564116             6.98             0.0   
4  4.434579e+10  11.194087             6.07             0.0   

   average_monthly_revenue  average_monthly_transactions  Median_age_persons  \
0                30.052613                  13786.333333           18.398239   
1             20772.116474                      0.333333           21.333333   
2                35.036088                  12984.476190           18.427831   
3                36.588292                  10869.285714           18.396191   
4             14546.024297                      4.238095           18.146667   

   Median_mortgage_repay_monthly  Median_rent_weekly  \
0                    1556.290754          299.626473   


In [182]:
merged_top_10_retail_goods.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,2.485245e+10,15.288695,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
1,6.414355e+10,14.735193,5.88,0.0,20772.116474,0.333333,21.333333,1131.000000,210.666667,1488.333333,Fusce Feugiat Associates,Retail Goods,0
2,8.657848e+10,14.441084,6.43,0.0,35.036088,12984.476190,18.427831,1560.985112,300.593985,1606.615735,Leo In Consulting,"Automotives, Retail Goods, Services",0
3,4.562922e+10,12.564116,6.98,0.0,36.588292,10869.285714,18.396191,1556.595595,299.956066,1605.239186,Lacus Consulting,"Automotives, Retail Goods",0
4,4.434579e+10,11.194087,6.07,0.0,14546.024297,4.238095,18.146667,1479.773333,278.840000,1632.093333,Phasellus Nulla LLC,Retail Goods,0


In [183]:
merged_top_10_media_technology.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,6.821691e+10,9.727362,3.05,0.0,38.604636,8610.714286,18.428784,1561.492133,300.728871,1606.650523,Placerat Eget Venenatis Limited,"Media and Technology, Furniture and Home Furni...",0
1,5.088570e+10,7.884283,2.84,0.0,37.960656,41.619048,18.185286,1558.359673,299.547684,1600.891008,Euismod Institute,"Media and Technology, Services",0
2,9.556074e+10,6.366724,6.22,0.0,32.105606,55.095238,18.375262,1535.715933,293.671908,1576.806080,Urna Nec Institute,"Retail Goods, Media and Technology",0
3,7.247291e+10,6.137252,6.33,0.0,70.011718,4847.190476,18.455684,1554.652268,299.542907,1603.243406,Nullam Consulting,"Retail Goods, Media and Technology",0
4,1.014225e+10,5.026798,4.22,0.0,37.716001,144.571429,18.285770,1566.667060,299.012972,1609.036557,Arcu Ac Orci Corporation,"Media and Technology, Services",0


In [184]:
merged_top_10_furniture_home_furnishing.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,2.485245e+10,15.288695,2.94,0.0,30.052613,13786.333333,18.398239,1556.290754,299.626473,1603.319408,Erat Vitae LLP,"Retail Goods, Furniture and Home Furnishing",0
1,6.420342e+10,13.890024,2.86,0.0,28.949356,12418.095238,18.419364,1562.089539,300.751919,1607.221653,Pede Nonummy Corp.,Furniture and Home Furnishing,0
2,4.989171e+10,13.388101,5.80,0.0,28.889602,11786.952381,18.410914,1558.776322,300.080589,1604.696067,Non Vestibulum Industries,Furniture and Home Furnishing,0
3,8.972601e+10,11.637619,6.01,0.0,41.329641,10283.952381,18.389066,1558.784247,300.248403,1606.079958,Est Nunc Consulting,Furniture and Home Furnishing,0
4,4.318652e+10,11.081336,4.47,0.0,44.812806,9567.285714,18.412755,1558.492916,300.077676,1605.109747,Lorem Ipsum Sodales Industries,"Retail Goods, Furniture and Home Furnishing",0


In [185]:
merged_top_10_services.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,3.915015e+10,37.199966,6.99,0.0,38089.939272,1.000000,11.000000,1408.000000,250.000000,1772.0,Aliquam Eu Institute,"Automotives, Services",0
1,1.040454e+10,20.685932,6.93,0.0,22773.975280,1.000000,17.000000,773.000000,170.000000,1412.0,Consequat Foundation,"Automotives, Services",0
2,2.906845e+10,17.434276,3.28,0.0,36950.787544,1.000000,37.000000,0.000000,0.000000,2749.0,Phasellus LLP,"Automotives, Services",0
3,5.486013e+10,15.967974,6.01,0.0,21520.803196,1.600000,18.357143,1546.928571,302.964286,1594.5,Laoreet Posuere Enim Limited,"Automotives, Services",0
4,5.142505e+10,15.016793,5.71,0.0,24582.746982,0.222222,31.000000,700.000000,200.000000,945.0,Sem Corporation,"Automotives, Services",0


In [186]:
merged_top_10_automotives.head()

,merchant_abn,score,take_rate_value,consumer_fraud,average_monthly_revenue,average_monthly_transactions,Median_age_persons,Median_mortgage_repay_monthly,Median_rent_weekly,Median_tot_hhd_inc_weekly,name,category_label,merchant_fraud
0,3.915015e+10,37.199966,6.99,0.0,38089.939272,1.000000,11.000000,1408.000000,250.000000,1772.0,Aliquam Eu Institute,"Automotives, Services",0
1,1.040454e+10,20.685932,6.93,0.0,22773.975280,1.000000,17.000000,773.000000,170.000000,1412.0,Consequat Foundation,"Automotives, Services",0
2,2.906845e+10,17.434276,3.28,0.0,36950.787544,1.000000,37.000000,0.000000,0.000000,2749.0,Phasellus LLP,"Automotives, Services",0
3,5.486013e+10,15.967974,6.01,0.0,21520.803196,1.600000,18.357143,1546.928571,302.964286,1594.5,Laoreet Posuere Enim Limited,"Automotives, Services",0
4,5.142505e+10,15.016793,5.71,0.0,24582.746982,0.222222,31.000000,700.000000,200.000000,945.0,Sem Corporation,"Automotives, Services",0
